In [1]:
import sqlalchemy
import psycopg2
import simplejson
%load_ext sql
 
%config SqlMagic.displaylimit = 5

In [2]:
with open("/Users/Neal/Desktop/Classpass/Analytics/redshift_creds.json.nogit") as fh:
    creds = simplejson.loads(fh.read())

In [3]:
connect_to_db = 'postgresql+psycopg2://' + \
                creds['user_name'] + ':' + creds['password'] + '@' + \
                creds['host_name'] + ':' + creds['port_num'] + '/' + creds['db_name'];
%sql $connect_to_db

u'Connected: npatel@prod'

In [4]:
from sqlalchemy import create_engine, text
conn_str = connect_to_db
engine = create_engine(conn_str)


In [55]:
import pandas as pd
sql = """WITH datetrunc AS(
  SELECT
    ao.user_id,
    ao.latitude,
    ao.longitude,
    date_trunc('minute', ao.server_timestamp) as timestamp
  FROM events2."iOS App Open" as ao),

time_lag AS(
  SELECT
    d.user_id,
    d.latitude,
    d.longitude,
    d.timestamp,
    lag(d.timestamp) OVER (PARTITION BY d.user_id ORDER BY d.timestamp ASC) as timestamp2
FROM datetrunc as d),

event_interval as(
  SELECT
    tl.user_id,
    tl.latitude,
    tl.longitude,
    tl.timestamp - tl.timestamp2 as interval
  FROM time_lag as tl),

event_case as(
  SELECT
    ei.user_id,
    ei.latitude,
    ei.longitude,
    CASE WHEN ei.interval > INTERVAL '5 mins' THEN 'new session'
      ELSE 'same session'
        END as session_cat
    FROM event_interval as ei)

SELECT
  ec.user_id,
  ec.latitude,
  ec.longitude
FROM event_case as ec
WHERE ec.session_cat = 'new session'
AND ec.user_id IS NOT NULL
AND ec.latitude IS NOT NULL"""
data = pd.read_sql(sql, engine)

In [7]:
##import gmplot
1

##gmap = gmplot.GoogleMapPlotter (0,0,2)
##heat_lats = data['latitude']
##heat_lngs = data['longitude']
##neal_map = gmap.heatmap(heat_lats, heat_lngs)
##gmap.draw("NealMap2.html")


In [56]:
from sklearn.cluster import DBSCAN
import multiprocessing
from multiprocessing import Pool
from datetime import datetime

db = DBSCAN(eps=.001, min_samples = 10)
user_locs = data.groupby(['user_id'])
user_ids = set(data['user_id'])
user_clusters = []

def user_cluster_points(id):
    startTime= datetime.now()
    user_data = user_locs.get_group(id)
    db.fit(user_data)
    user_data['cluster'] = db.labels_
    user_data = user_data.groupby('cluster').mean()
    return user_data
    time = datetime.now() - startTime

if __name__ == '__main__':
    pool = Pool(processes=6)
    inputs = user_ids
    results = pool.map(user_cluster_points, inputs)
    pool.close()
    pool.join()

# while (True):
#   completed = results._index
#   num_tasks = len(user_ids)
#   if (completed == num_tasks): break
#   print "Waiting for", num_tasks-completed, "tasks to complete..."
#   time.sleep(2)

# if __name__ == '__main__':
#     jobs = []
#     for id in user_ids:
#         p = multiprocessing.Process(target=user_cluster_points)
#         jobs.append(p)
#         p.start()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [53]:
# def user_cluster_points(id): 
#     startTime= datetime.now()
#     user_data = user_locs.get_group(id)
#     db.fit(user_data)
#     user_data['cluster'] = db.labels_
#     user_data = user_data.groupby('cluster').mean()
#     user_clusters.append(user_data)
#     time = datetime.now() - startTime
    
# for id in user_ids:
#     user_cluster_points(id)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [58]:
results

[         user_id  latitude  longitude
 cluster                              
 0         262145       0.0        0.0,          user_id   latitude  longitude
 cluster                               
 -1        961195  39.426263 -76.966639
  0        961195  40.747024 -74.029073
  1        961195  40.741376 -73.989704,          user_id   latitude  longitude
 cluster                               
 -1        786436  41.019185 -73.589593
  0        786436  40.772092 -73.954502,          user_id  latitude  longitude
 cluster                              
 -1        524293 -33.89391   151.1727,          user_id   latitude  longitude
 cluster                               
 -1        917505  40.804091  -73.96333,          user_id   latitude   longitude
 cluster                                
 -1       1066095  47.686949 -122.259158,          user_id   latitude   longitude
 cluster                                
 -1        786443 -32.902619  151.694412
  0        786443 -33.891019  151.283311

In [61]:
user_clusters = pd.concat(results)
user_clusters.to_csv("~/Desktop/user_clusters.csv")